# Woe Binning

Курс: "Матетичне моделювання в R"

---

In [ ]:
# install.packages("openxlsx")

## Набір даних

Джерело: <https://github.com/gastonstat/CreditScoring/blob/master/CleanCreditScoring.csv>

Завантажимо дані:

In [ ]:
library(openxlsx)
data <- openxlsx::read.xlsx("data/CreditScoring.xlsx", sheet = 1, startRow = 1, colNames = TRUE, rowNames = FALSE)
str(data)

Опишемо дані:

- [x] `Status` - credit status (`Target`)
- [x] `Seniority` job seniority (years)
- [x] `Home` type of home ownership
- [x] `Time` time of requested loan
- [x] `Age` client’s age
- [x] `Marital` marital status
- [x] `Records` existance of records
- [x] `Job` type of job
- [x] `Expenses` amount of expenses
- [x] `Income` amount of income
- [x] `Assets` amount of assets
- [x] `Debt` amount of debt
- [x] `Amount` amount requested of loan
- [x] `Price` price of good

---

## Простий бінінг змінних

### Категоріальна змінна

Створимо дата-фрейм для зберігання інформації про групи змінної `Home` (як приклад біннігу категоріального показника):

In [ ]:
home_groups <- data.frame(Group = unique(data$Home), 
                          Good = c(0), Bad = c(0), 
                          GoodP = c(0), BadP = c(0),
                          WOE = c(0), IV = c(0))
home_groups

Переглянемо можливі варіанти показника `Status`, який є залежною бінарною змінною поточної задачі:

In [ ]:
library(gmodels)
CrossTable(data$Status)

Сформуємо групи та обчислимо значення по кожній групі:

In [ ]:
for(i in 1:nrow(home_groups)) {
  
  group <- home_groups$Group[i]
  
  home_groups$Good[i] <- nrow(data[data$Home == group & data$Status == "good", ])
  home_groups$Bad[i] <- nrow(data[data$Home == group & data$Status == "bad", ])
  
  home_groups$GoodP[i] <- home_groups$Good[i]/nrow(data[data$Status == "good", ])
  home_groups$BadP[i] <- home_groups$Bad[i]/nrow(data[data$Status == "bad", ])
  
  home_groups$WOE[i] <- log( home_groups$GoodP[i] / home_groups$BadP[i])
  home_groups$IV[i] <- (home_groups$GoodP[i] - home_groups$BadP[i])*home_groups$WOE[i]
}

home_groups

Переглянемо сумарний `IV`:

In [ ]:
home_iv <- sum(home_groups$IV)
home_iv

Візуалізуємо групи:

In [ ]:
barplot(home_groups$WOE, 
        col="brown", 
        names.arg=c(as.character(home_groups$Group)), 
        xlab="Group",
        ylab="WOE"
)

Створимо датафрейм для нових `WOE`-даних:

In [ ]:
new_df <- data.frame(Status = data$Status, Home = data$Home, HomeWoe = c(0))

Замінимо значення на `WOE`:

In [ ]:
for(i in 1:nrow(home_groups)) {
  group <- home_groups$Group[i]
  woe <- home_groups$WOE[i]  
  new_df[new_df$Home == group, ]$HomeWoe <- woe
}

tail(new_df)

---

### Числова змінна

Обрахуємо приклад числовиго показника (на прикладі `Age`):

In [ ]:
min_age <- min(data$Age)
max_age <- max(data$Age)

step <- round(max_age - min_age)/10


age_groups <- data.frame(Min = seq(min_age, max_age-step, step), 
                         Max = seq(min_age + step, max_age, step), 
                          Good = c(0), Bad = c(0), 
                          GoodP = c(0), BadP = c(0),
                          WOE = c(0), IV = c(0))

age_groups

Сформуємо групи:

In [ ]:
for(i in 1:nrow(age_groups)) {
  
  min <- age_groups$Min[i]
  max<- age_groups$Max[i]
  
  age_groups$Good[i] <- nrow(data[data$Age >= min & data$Age < max & data$Status == "good", ])
  age_groups$Bad[i] <- nrow(data[data$Age >= min & data$Age < max & data$Status == "bad", ])
  
  if(i == nrow(age_groups)) {
    age_groups$Good[i] <- age_groups$Good[i] + nrow(data[data$Age == max & data$Status == "good", ])
    age_groups$Bad[i] <- age_groups$Bad[i] + nrow(data[data$Age == max & data$Status == "bad", ])
  }
  
  age_groups$GoodP[i] <- age_groups$Good[i]/nrow(data[data$Status == "good", ])
  age_groups$BadP[i] <- age_groups$Bad[i]/nrow(data[data$Status == "bad", ])
  
  age_groups$WOE[i] <- log( age_groups$GoodP[i] / age_groups$BadP[i])
  
  age_groups$IV[i] <- (age_groups$GoodP[i] - age_groups$BadP[i]) * age_groups$WOE[i]
}

age_groups

Сумарний IV:

In [ ]:
age_iv <- sum(age_groups$IV)
age_iv

Візуалізуємо значення груп `WOE`:

In [ ]:
barplot(age_groups$WOE, 
        col="brown", 
        names.arg=c(age_groups$Min), 
        xlab="Min Age",
        ylab="WOE"
)

Замінимо значення на `WOE`:

In [ ]:
new_df$Age <- data$Age
new_df$AgeWoe <- c(0)

for(i in 1:nrow(age_groups)) {
  
  min <- age_groups$Min[i]
  max <- age_groups$Max[i]
  woe <- age_groups$WOE[i]
  
  new_df[new_df$Age >= min & new_df$Age < max, ]$AgeWoe <- woe
  
  if(i == nrow(age_groups)) {
    new_df$AgeWoe[i] <- woe
  }
  
}

head(new_df)

Видалимо оригінальні значення з набору даних:

In [ ]:
new_df$Home <- NULL
new_df$Age <- NULL
head(new_df)

---

## Бінінг з використанням `woeBin`

---

## Бінінг з використанням `rBin`

---

## Бінінг з використанням `smbinning`

---

## Бінінг з використанням `scorecard`

---